In [124]:
# Step 1, you must install Selenuim
#pip install selenium

In [184]:
# once that is installed, install Chrome Driver (in accordance with your Chrome Browser's version)
# and put that file (chromedriver.exe) in your C:\Program Files (x86) for ease of access

In [186]:
# How to use, just run this line and give it a movie name, it will serach that on Kids in Mind and will fetch
# the related info
title = input("Tell me some english movie name:")
grabMoviesFromKIM(title)

Tell me some english movie name:Wrong Turn
Wrong Turn [2003] [R] - 3.10.5
<re.Match object; span=(24, 30), match='3.10.5'>
The Campaign [2012] [R] - 7.5.7
<re.Match object; span=(26, 31), match='7.5.7'>
Leap Year [2009] [PG] - 3.3.4
<re.Match object; span=(24, 29), match='3.3.4'>
The Amazing Spider-Man [2012] [PG-13] - 3.6.4
<re.Match object; span=(40, 45), match='3.6.4'>
The Sorcerer's Apprentice [2010] [PG] - 2.5.2
<re.Match object; span=(40, 45), match='2.5.2'>
Ford v Ferrari [2019] [PG-13] - 2.5.5
<re.Match object; span=(32, 37), match='2.5.5'>
Lucy [2014] [R] - 4.7.4
<re.Match object; span=(18, 23), match='4.7.4'>
Armored [2009] [PG-13] - 1.6.5
<re.Match object; span=(25, 30), match='1.6.5'>
Black Sea [2015] [R] - 2.6.10
<re.Match object; span=(23, 29), match='2.6.10'>
Fun with Dick and Jane [2005] [PG-13] - 3.3.5
<re.Match object; span=(40, 45), match='3.3.5'>
The Hunger Games: Mockingjay - Part 2 [2015] [PG-13] - 2.6.1
<re.Match object; span=(55, 60), match='2.6.1'>
It's Complic

In [188]:
def evalChunks(txt):
    infoList = {}
    m = re.search(r"\[([0-9]+)\]", txt)
    if not m:
        year = 'N/a'
    else:
        year = m.group(1)
    bm = re.search(r'\d+.\d+.\d+', txt)
    if not bm:
        bm = 'N/a'
    else:
        bm = bm.group()

    # movie name
    movie = txt.replace('['+year+']','').replace('-','').replace(bm,'')
    rating = re.search(r"\[(.*?)\]", movie)
    if not rating:
        rating = 'N/a'
    else:
        rating = rating.group(1)
    movie = movie.replace('['+rating+']','')
    if not bm == 'N/a':
        bm = bm.split('.')
        infoList['Vulgarity'] = bm[0]
        infoList['Violence'] = bm[1]
        infoList['Profanity'] = bm[2]        
    else:
        infoList['Vulgarity'] = ''
        infoList['Violence'] = ''
        infoList['Profanity'] = ''    
    infoList['Movie'] = movie.strip(' ')
    infoList['Year'] = year
    infoList['Rating'] = rating

    return infoList

In [181]:
def writeToFile(df_new):
    import pandas as pd
    import os.path
    path = './movies.csv'
    check_file = os.path.isfile(path)
    if not check_file:
        df_new.to_csv('movies.csv',index=False)
    else:
        df_existing = pd.read_csv('movies.csv')
        df_existing.reset_index(drop=True, inplace=True)
        df_new.reset_index(drop=True, inplace=True)
        concatDF = pd.concat([df_existing, df_new]).drop_duplicates()
        concatDF.to_csv('movies.csv',index=False)
    print("Records written to file successfully.");

In [187]:
def grabMoviesFromKIM(title):
    print("Please wait for the SUCCESS message or ERRORS otherwise.")
    print("If you see the success message, then load the new created file 'movies.csv'")
    from selenium.webdriver.chrome.service import Service
    from selenium import webdriver 
    from selenium.webdriver.common.by import By
    import re 
    import pandas as pd
    path = "C:\Program Files (x86)\chromedriver.exe"
    service = Service(executable_path=path)
    driver = webdriver.Chrome(service=service)
    driver.get('https://kids-in-mind.com/search-desktop.htm')
    driver.find_element(By.ID, 's').send_keys(title)
    driver.find_element(By.ID, 'searchsubmit').click()
    driver.implicitly_wait(5)
    sr = driver.find_elements(By.CLASS_NAME,'search-result')
    movies = []
    for span in sr:
        movies.append(evalChunks(span.text))  
    df = pd.DataFrame(movies)  
    df.reset_index(drop=True)
    writeToFile(df)